In [51]:
#!/usr/bin/env python

import numpy as np
import pandas as pd


nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))

nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    
    if nt < 30:  # We need at least 30 days of data
        return np.zeros(nins)
    
    # Calculate z-score of prices
    lookback = 11
    means = np.mean(prcSoFar[:, -lookback:], axis=1)
    stds = np.std(prcSoFar[:, -lookback:], axis=1)
    z_scores = (prcSoFar[:, -1] - means) / stds
    
    # Apply z-score threshold
    z_score_threshold = 0.5  # Adjust this value as needed
    signals = np.zeros(nins)
    signals[z_scores > z_score_threshold] = -1  # Short when z-score is high
    signals[z_scores < -z_score_threshold] = 1  # Long when z-score is low
    
    # Normalize signals
    if np.sum(np.abs(signals)) > 0:
        signals = signals / np.sum(np.abs(signals))
    
    # Calculate target positions
    target_position = np.array([int(x) for x in dlrPosLimit * 0.5 * signals / prcSoFar[:, -1]])
    
    # Implement stop loss
    stop_loss_percentage = 0.01  # 1% stop loss
    entry_prices = prcSoFar[:, -2]  # Assuming we entered the position yesterday
    current_prices = prcSoFar[:, -1]
    
    for i in range(nins):
        if currentPos[i] > 0:  # Long position
            if current_prices[i] < entry_prices[i] * (1 - stop_loss_percentage):
                target_position[i] = 0  # Close the position
        elif currentPos[i] < 0:  # Short position
            if current_prices[i] > entry_prices[i] * (1 + stop_loss_percentage):
                target_position[i] = 0  # Close the position
    
    # Implement a conservative position change
    position_change_rate_range = 0.1
    position_change = position_change_rate_range * (target_position - currentPos)
    
    # Update current position
    currentPos = np.array([int(x) for x in currentPos + position_change])
    
    # Ensure we're within position limits
    posLimits = np.array([int(x) for x in dlrPosLimit / current_prices])
    currentPos = np.clip(currentPos, -posLimits, posLimits)
    
    return currentPos

# The rest of the code (calcPL function and the main execution) remains the same

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(500, 751):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        print(newPos)
        if (totDVolume > 0):
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd


print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

Loaded 50 instruments for 750 days
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
Day 500 value: -0.01 todayPL: $-0.01 $-traded: 11 return: -0.00100
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0]
Day 501 value: -0.06 todayPL: $-0.04 $-traded: 55 return: -0.00100
[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
Day 502 value: 0.07 todayPL: $0.13 $-traded: 88 return: 0.00082
[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
Day 503 value: -0.13 todayPL: $-0.20 $-traded: 99 return: -0.00130
[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
Day 504 value: -0.21 todayPL: $-0.08 $-traded: 109 return: -0.00191
[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
Day 505 value: